# Team 순풍산부인과
### 실행환경 코랩(Colab)
### 최종 사용모델
WeightedEnsemble_L2  
### 사용라이브러리
asttokens==3.0.0  
autogluon==1.2  
backcall==0.2.0  
backports.tarfile==1.2.0  
certifi==2025.1.31  
colorama==0.4.6  
cycler==0.12.1  
debugpy==1.8.0  
decorator==4.4.2  
defusedxml==0.7.1  
executing==2.2.0  
google==2.0.3  
httplib2==0.22.0  
ipykernel==6.17.1  
jaraco.context==5.3.0  
jaraco.functools==4.0.1  
jaraco.text==3.12.1  
jedi==0.19.2  
kiwisolver==1.4.8  
matplotlib==3.10.0  
numpy==1.26.4  
packaging==24.2  
parso==0.8.4   
pathlib==1.0.1  
pexpect==4.9.0  
pickleshare==0.7.5  
platformdirs==4.3.6   
portpicker==1.5.2  
psutil==5.9.5  
ptyprocess==0.7.0  
pygments==2.18.0  
pyparsing==3.2.1  
six==1.17.0   
tornado==6.4.2   
traitlets==5.7.1   
wcwidth==0.2.13  
zipp==3.21.0  

In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. 

코랩 전용 저장소및 드라이브 설정

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/AIMERS/

/content/drive/MyDrive/AIMERS


In [4]:
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

In [5]:
import ray

# Ray 초기화 시 CPU 수를 최대한 활용하도록 설정
ray.shutdown()  # 기존 세션 종료
ray.init(num_cpus=8)  # 현재 Colab에서 사용 가능한 최대 CPU로 설정

print(f"사용 가능한 CPU 개수: {ray.available_resources().get('CPU', 0)}")


2025-03-04 08:36:02,081	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


사용 가능한 CPU 개수: 8.0


In [6]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [ ]:
#########################################
#########################################
def preprocess_data(df):

    # 제거할 컬럼
    drop_columns = [
        '불임 원인 - 여성 요인',
        '불임 원인 - 자궁경부 문제', '미세주입 후 저장된 배아 수', '불임 원인 - 정자 면역학적 요인',
        '불임 원인 - 정자 운동성', '시술 유형', '난자 해동 경과일', 'DI 출산 횟수', '저장된 신선 난자 수',
        '정자 출처', '임신 시도 또는 마지막 임신 경과 연수',
        '부부 부 불임 원인', '여성 부 불임 원인', '불임 원인 - 정자 형태', '대리모 여부', '불임 원인 - 정자 농도',
        'DI 임신 횟수', '기증 배아 사용 여부', '착상 전 유전 진단 사용 여부', '미세주입 배아 이식 수',
        '부부 주 불임 원인', '기증자 정자와 혼합된 난자 수', '배아 해동 경과일', 'PGD 시술 여부', '파트너 정자와 혼합된 난자 수',
        '혼합된 난자 수', '불임 원인 - 자궁경부 문제'
    ]

    df = df.drop(columns=[col for col in drop_columns if col in df.columns])

    # ✅ 피처 엔지니어링 적용
    df["배아 수 제곱"] = df["이식된 배아 수"] ** 2



    # ✅ 연령대를 중앙값으로 변환하여 숫자로 변환
    age_mapping = {
        "만18-34세": 26,
        "만35-37세": 36,
        "만38-39세": 38.5,
        "만40-42세": 41,
        "만43-44세": 43.5,
        "만45-50세": 47.5,
        "알 수 없음": -1  # "알 수 없음"을 특별한 값으로 처리
    }

    # ✅ 매핑 적용
    df["시술 당시 나이 숫자"] = df["시술 당시 나이"].map(age_mapping)

    # 배아 개수 대비 저장된 배아 수 비율 (배아 보존율)
    df["배아 보존율"] = df["저장된 배아 수"] / (df["총 생성 배아 수"] + 1)  # 0으로 나누는 것 방지

    # ✅ 나이와 배아 수의 비율 변수 생성
    df["배아 수 대비 나이"] = df["시술 당시 나이 숫자"] / (df["이식된 배아 수"] + 1)

    # ✅ 추가 비율 Feature 생성
    df["배아 이식 경과일 대비 나이"] = df["배아 이식 경과일"] / (df["시술 당시 나이 숫자"] + 1)

    df["1~2개 이식 여부"] = (df["이식된 배아 수"].between(1, 2)).astype(int)

    df["나이 26~36 여부"] = ((df["시술 당시 나이 숫자"] >= 26) &
                      (df["시술 당시 나이 숫자"] <= 36)).astype(int)

    # 1️⃣ 이식된 배아 수 × 시술 당시 나이 숫자
    df["이식된 배아 수 × 시술 당시 나이"] = df["이식된 배아 수"] * df["시술 당시 나이 숫자"]

    # 2️⃣ 배아 이식 경과일 대비 나이 × 배아 수 대비 나이
    df["배아 이식 경과일 대비 나이 × 배아 수 대비 나이"] = df["배아 이식 경과일 대비 나이"] * df["배아 수 대비 나이"]

    # 3️⃣ 배아 수 대비 나이 / 총 생성 배아 수
    df["배아 수 대비 나이 / 총 생성 배아 수"] = df["배아 수 대비 나이"] / (df["총 생성 배아 수"] + 1)

    # 4️⃣ 이식된 배아 수 × 배아 이식 경과일 대비 나이
    df["이식된 배아 수 × 배아 이식 경과일 대비 나이"] = df["이식된 배아 수"] * df["배아 이식 경과일 대비 나이"]

    # 5️⃣ 시술 당시 나이 숫자 / 총 생성 배아 수
    df["시술 당시 나이 숫자 / 총 생성 배아 수"] = df["시술 당시 나이 숫자"] / (df["총 생성 배아 수"] + 1)

    return df

In [ ]:
# ✅ 데이터 로드
train = pd.read_csv("data/train.csv", encoding="utf-8")
test = pd.read_csv("data/test.csv", encoding="utf-8")

In [ ]:
train = preprocess_data(train)
test = preprocess_data(test)

In [ ]:
# ✅ ID 제거
train = train.drop(columns=["ID"])
test_id = test["ID"]
test= test.drop(columns=["ID"])

In [ ]:
# ✅ 모델 저장 경로
save_path = "FINAL_MODEL"

In [ ]:
hyperparameters = {
    "GBM": {},
    "CAT": {},
    "XGB": {},
    "RF": {},
    "FASTAI": {},
    "XT": {},
    "NN_TORCH": {}
}

In [ ]:

# 📌 2️⃣ 모델 학습 설정
predictor = TabularPredictor(label="임신 성공 여부", eval_metric="roc_auc", path=save_path)

predictor.fit(
    train_data=train,
    presets="best_quality",
    time_limit=3600*12,  # 학습 제한 시간 (초)
    num_bag_folds=8,  # Bagging 적용
    num_stack_levels=0,  # Stacking L2
    dynamic_stacking=False,
    save_space=True,
    hyperparameters = hyperparameters,
    hyperparameter_tune_kwargs = {
        "num_trials": 20,  # 🚀 각 모델당 20번의 랜덤 탐색
        "scheduler": "local",
        "searcher": "random"
    }
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          8
Memory Avail:       42.85 GB / 50.99 GB (84.0%)
Disk Space Avail:   139.61 GB / 200.00 GB (69.8%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "/content/drive/MyDrive/AIMERS/FINAL_MODEL"
Train Data Rows:    256351
Train Data Columns: 53
Label Column:       임신 성공 여부
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as 

  0%|          | 0/20 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.80%)
(_ray_fit pid=20480) 	Ran out of time, early stopping on iteration 234.
	Stopping HPO to satisfy time limit...
Fitted model: LightGBM_BAG_L1/T1 ...
	0.7398	 = Validation score   (roc_auc)
	312.3s	 = Training   runtime
	4.63s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L1 ... Tuning model for up to 230.77s of the 1481.47s of remaining time.
	No hyperparameter search space specified for RandomForest_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: RandomForest_BAG_L1 ...
	0.7293	 = Validation score   (roc_auc)
	61.66s	 = Training   runtime
	16.82s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ... Tuning model for up to 230.77s of the 1417.42s of remaining time.


  0%|          | 0/20 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.75%)
(_ray_fit pid=23394) 	Ran out of time, early stopping on iteration 113. [repeated 8x across cluster]
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L1/T1 ...
	0.7379	 = Validation score   (roc_auc)
	181.94s	 = Training   runtime
	0.5s	 = Validation runtime
Hyperparameter tuning model: ExtraTrees_BAG_L1 ... Tuning model for up to 230.77s of the 1234.62s of remaining time.
	No hyperparameter search space specified for ExtraTrees_BAG_L1. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: ExtraTrees_BAG_L1 ...
	0.7316	 = Validation score   (roc_auc)
	51.73s	 = Training   runtime
	16.54s	 = Validation runtime
Hyperparameter tuning model: NeuralNetFastAI_BAG_L1 ... Tuning model for up to 230.77s of the 1181.23s of remaining time.


...

  0%|          | 0/20 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.05%)
	Stopping HPO to satisfy time limit...
Fitted model: XGBoost_BAG_L1/T1 ...
	0.7388	 = Validation score   (roc_auc)
	187.26s	 = Training   runtime
	3.42s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 230.77s of the 750.82s of remaining time.


+----------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch_BAG_L1   |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        FIFOScheduler           |
| Number of trials                 20                      |
+----------------------------------------------------------+

View detailed results here: /content/drive/MyDrive/AIMERS/FINAL_MODEL/models/NeuralNetTorch_BAG_L1


(_ray_fit pid=28565) 	Ran out of time, stopping training early. (Stopping on epoch 15) [repeated 8x across cluster]
(_ray_fit pid=29874) 	Ran out of time, stopping training early. (Stopping on epoch 1) [repeated 8x across cluster]
2025-02-28 10:01:27,182	INFO timeout.py:54 -- Reached timeout of 230.77219184466773 seconds. Stopping all trials.
2025-02-28 10:01:27,351	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/content/drive/MyDrive/AIMERS/FINAL_MODEL/models/NeuralNetTorch_BAG_L1' in 0.1016s.
2025-02-28 10:01:32,078	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 20 trial(s):
- 6fca7_00000: FileNotFoundError('Could not fetch metrics for 6fca7_00000: both result.json and progress.csv were not found at /content/drive/MyDrive/AIMERS/FINAL_MODEL/models/NeuralNetTorch_BAG_L1/6fca7_00000')
- 6fca7_00001: FileNotFoundError('Could not fetch metrics for 6fca7_00001: both result.json and progress.csv were not found at /content/drive

Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 513.82s of remaining time.
	Ensemble Weights: {'LightGBM_BAG_L1/T1': 0.524, 'NeuralNetTorch_BAG_L1/6fca7_00000': 0.19, 'NeuralNetFastAI_BAG_L1/6f3c9_00000': 0.143, 'XGBoost_BAG_L1/T1': 0.095, 'RandomForest_BAG_L1': 0.048}
	0.7404	 = Validation score   (roc_auc)
	11.95s	 = Training   runtime
	0.05s	 = Validation runtime
AutoGluon training complete, total runtime = 1298.59s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1416.0 rows/s (32044 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/drive/MyDrive/AIMERS/FINAL_MODEL")


In [ ]:
# 📌 3️⃣ 예측 수행
pred = predictor.predict_proba(test)
submission = pd.DataFrame({"ID": test_id, "probability": pred[1]})

In [ ]:
# 결과 병합 및 저장
submission.to_csv("data/FINAL_Submission.csv", index=False, encoding="utf-8")

print("Submission file created: FINAL_Submission.csv")

Submission file created: FINAL_Submission.csv
